In [1]:
import csv
import json
import os
from datetime import datetime

# 1. Product Management (CSV File)

def add_product(csv_file, product):
    try:
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(product)
    except FileNotFoundError:
        print(f"Error: The file {csv_file} does not exist.")
    except PermissionError:
        print(f"Error: Permission denied to write to {csv_file}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def update_product(csv_file, product_id, new_price, new_stock):
    try:
        updated = False
        rows = []
        with open(csv_file, mode='r', newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[0] == product_id:
                    row[3] = new_price  # Update price
                    row[4] = new_stock  # Update stock quantity
                    updated = True
                rows.append(row)

        if updated:
            with open(csv_file, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(rows)
        else:
            print(f"Product ID {product_id} not found.")
    except FileNotFoundError:
        print(f"Error: The file {csv_file} does not exist.")
    except PermissionError:
        print(f"Error: Permission denied to write to {csv_file}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def check_product_availability(csv_file, product_id):
    try:
        with open(csv_file, mode='r', newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[0] == product_id:
                    stock = int(row[4])
                    if stock > 0:
                        print(f"Product {row[1]} is in stock with {stock} units available.")
                    else:
                        print(f"Product {row[1]} is out of stock.")
                    return
            print(f"Product ID {product_id} not found.")
    except FileNotFoundError:
        print(f"Error: The file {csv_file} does not exist.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# 2. Customer Review Management (JSON Files)

def add_customer_review(product_id, review):
    filename = f"{product_id}_reviews.json"
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                reviews = json.load(file)
        else:
            reviews = []

        reviews.append(review)

        with open(filename, 'w') as file:
            json.dump(reviews, file, indent=4)
    except FileNotFoundError:
        print(f"Error: The file {filename} does not exist.")
    except json.JSONDecodeError:
        print(f"Error: The file {filename} contains corrupted data.")
    except PermissionError:
        print(f"Error: Permission denied to write to {filename}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def calculate_average_rating(product_id):
    filename = f"{product_id}_reviews.json"
    try:
        with open(filename, 'r') as file:
            reviews = json.load(file)
            total_rating = sum(review['Rating'] for review in reviews)
            average_rating = total_rating / len(reviews) if reviews else 0
            print(f"Average rating for Product ID {product_id}: {average_rating:.2f}")
    except FileNotFoundError:
        print(f"Error: The file {filename} does not exist.")
    except json.JSONDecodeError:
        print(f"Error: The file {filename} contains corrupted data.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def search_reviews(product_id, keyword):
    filename = f"{product_id}_reviews.json"
    try:
        with open(filename, 'r') as file:
            reviews = json.load(file)
            found_reviews = [review for review in reviews if keyword.lower() in review['Comment'].lower()]
            for review in found_reviews:
                print(review)
            if not found_reviews:
                print("No reviews found containing the keyword.")
    except FileNotFoundError:
        print(f"Error: The file {filename} does not exist.")
    except json.JSONDecodeError:
        print(f"Error: The file {filename} contains corrupted data.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# 3. Data Analysis

def top_rated_products(reviews_folder):
    product_ratings = {}
    try:
        for filename in os.listdir(reviews_folder):
            if filename.endswith('_reviews.json'):
                with open(os.path.join(reviews_folder, filename), 'r') as file:
                    reviews = json.load(file)
                    average_rating = sum(review['Rating'] for review in reviews) / len(reviews) if reviews else 0
                    product_id = filename.split('_')[0]
                    product_ratings[product_id] = average_rating
        
        top_products = sorted(product_ratings.items(), key=lambda x: x[1], reverse=True)[:5]
        for product in top_products:
            print(f"Product ID: {product[0]}, Average Rating: {product[1]:.2f}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def out_of_stock_products(csv_file):
    try:
        out_of_stock = []
        with open(csv_file, mode='r', newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                if int(row[4]) == 0:  # Stock Quantity
                    out_of_stock.append(row[1])  # Product Name
        
        if out_of_stock:
            print("Out of stock products:")
            for product in out_of_stock:
                print(product)
        else:
            print("All products are in stock.")
    except FileNotFoundError:
        print(f"Error: The file {csv_file} does not exist.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def price_trends(csv_file):
    try:
        changes = []
        with open(csv_file, mode='r', newline='') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            for row in reader:
                # Assuming we have a way to track previous prices, e.g., via a separate CSV
                previous_price = get_previous_price(row[0])  # Placeholder for actual implementation
                if previous_price:
                    current_price = float(row[3])
                    if current_price > previous_price:
                        changes.append((row[1], 'increased', previous_price, current_price))
                    elif current_price < previous_price:
                        changes.append((row[1], 'decreased', previous_price, current_price))
        
        for change in changes:
            print(f"Product {change[0]} price has {change[1]} from {change[2]} to {change[3]}.")
    except FileNotFoundError:
        print(f"Error: The file {csv_file} does not exist.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_previous_price(product_id):
    # Placeholder function to get the previous price of a product
    # Implementation would depend on your data storage method
    return None

# Example Usage
if __name__ == "__main__":
    # Add a new product
    add_product('inventory.csv', ['101', 'Laptop', 'Electronics', '1200', '50'])
    
    # Update product
    update_product('inventory.csv', '101', '1150', '45')

    # Check availability
    check_product_availability('inventory.csv', '101')

    # Add a review
    add_customer_review('101', {
        'Review ID': '1',
        'User ID': 'User1',
        'Rating': 5,
        'Comment': 'Excellent product!',
        'Date': datetime.now().isoformat()
    })

    # Calculate average rating
    calculate_average_rating('101')

    # Search reviews
    search_reviews('101', 'Excellent')

    # Top rated products
    top_rated_products('./reviews')

    # Out of stock products
    out_of_stock_products('inventory.csv')

    # Price trends
    price_trends('inventory.csv')


Product Laptop is in stock with 45 units available.
Average rating for Product ID 101: 5.00
{'Review ID': '1', 'User ID': 'User1', 'Rating': 5, 'Comment': 'Excellent product!', 'Date': '2024-10-11T10:13:27.896972'}
An unexpected error occurred: [WinError 3] The system cannot find the path specified: './reviews'
All products are in stock.
